<a href="https://colab.research.google.com/github/LeninGF/CoursesNotes/blob/main/InteligenciaArtificalGenerativa/Problems/LSTM-GRU/EjercicioEmbeddingLSTM-GRU-IAG-2024B_LeninFalconi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Multiclase

Coder: Lenin G. Falconí

Asignatura: Tópicos Especiales (Inteligencia Artificial)

Fecha: 2024-11-18

## Generación de Texto 
Usando la información presentada, programar el código para la generación de texto basado en las frases de Sheldon para ejecutar en Kaggle. 
Incluir la preparación de datos, definición del modelo, entrenamiento y generación de texto. Ajustar para mejorar resultados, comparar generación por caracteres vs. Generación por palabras. Probar para otros vocabularios. 


In [1]:
chars_window = 20  # Tamaño de la ventana deslizante
step = 3           # Paso para mover la ventana
n_epochs = 5      # Número de épocas (ajustable)
batch_size = 128   # Tamaño del lote

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

# Load dataset
path_to_file = tf.keras.utils.get_file('don_quijote.txt', 'https://www.gutenberg.org/files/2000/2000-0.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Create a vocabulary
vocab = sorted(set(text))
char_to_index = {char: index for index, char in enumerate(vocab)}
index_to_char = np.array(vocab)

# Convert text to integers
text_as_int = np.array([char_to_index[char] for char in text])

2226045/2226045 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
len(vocab)

106

In [4]:
import pandas as pd
sentences = []
next_chars = []
for sentence in text.split('\n'):
    for i in range(0, len(sentence) - chars_window, step):
        sentences.append(sentence[i:i+chars_window])
        next_chars.append(sentence[i+chars_window])
df = pd.DataFrame({'sentence': sentences, 'next_char':next_chars})
df.sample(5)

,sentence,next_char
139126,e prometido a la pri,n
237379,¿Qué lobos os espan,t
149585,"golpes y brazos, pro",b
207959,que no debéis dar cu,l
224267,de preguntar qué sig,n


## Copilot

In [5]:
text_as_int[:3]

array([105,  49,  65])

Preparar datos

In [6]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# Create training examples and targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [7]:
cont = 0
for i in dataset:
    while(cont<10):
        print(i)
        cont+=1

(<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([105,  49,  65,  62,   2,  45,  75,  72,  67,  62,  60,  77,   2,
        36,  78,  77,  62,  71,  59,  62,  75,  64,   2,  62,  31,  72,
        72,  68,   2,  72,  63,   2,  33,  72,  71,   2,  46,  78,  66,
        67,  72,  77,  62,  12,   2,  59,  82,   2,  42,  66,  64,  78,
        62,  69,   2,  61,  62,   2,  32,  62,  75,  79,  58,  71,  77,
        62,  76,   2,  48,  58,  58,  79,  62,  61,  75,  58,   1,   0,
         1,   0,  49,  65,  66,  76,   2,  62,  31,  72,  72,  68,   2,
        66,  76,   2,  63,  72,  75,   2,  77,  65])>, <tf.Tensor: shape=(100,), dtype=int64, numpy=
array([49, 65, 62,  2, 45, 75, 72, 67, 62, 60, 77,  2, 36, 78, 77, 62, 71,
       59, 62, 75, 64,  2, 62, 31, 72, 72, 68,  2, 72, 63,  2, 33, 72, 71,
        2, 46, 78, 66, 67, 72, 77, 62, 12,  2, 59, 82,  2, 42, 66, 64, 78,
       62, 69,  2, 61, 62,  2, 32, 62, 75, 79, 58, 71, 77, 62, 76,  2, 48,
       58, 58, 79, 62, 61, 75, 58,  1,  0,  1,  

In [8]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Modelo

In [9]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_shape=(seq_length,)),
        LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size=len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 256)       │        27,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100, 1024)      │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100, 106)       │       108,650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,382,762 (20.53 MB)

 Trainable params: 5,382,762 (20.53 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)


In [12]:
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS)


Epoch 1/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 26s 65ms/step - loss: 2.5158
Epoch 2/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 24s 68ms/step - loss: 1.5839
Epoch 3/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 26s 72ms/step - loss: 1.3724
Epoch 4/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 28s 78ms/step - loss: 1.2792
Epoch 5/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 29s 81ms/step - loss: 1.2218
Epoch 6/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 28s 79ms/step - loss: 1.1842
Epoch 7/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 28s 79ms/step - loss: 1.1424
Epoch 8/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 29s 81ms/step - loss: 1.1143
Epoch 9/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 28s 80ms/step - loss: 1.0881
Epoch 10/10
335/335 ━━━━━━━━━━━━━━━━━━━━ 28s 80ms/step - loss: 1.0596


Generar Texto

In [13]:
def generate_text(model, start_string, num_generate=10, temperature=1.0):
    num_generate = num_generate
    input_eval = [char_to_index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = temperature

#     model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(index_to_char[predicted_id])

    return start_string + ''.join(text_generated)


In [14]:
print(generate_text(model, start_string="panza, como vas?", temperature=1, num_generate=200))

panza, como vas? llvichetos os uijeseratadelioncimardiamo ra dire
y Esterestodió sente es múndo dene tues leltaselore rtos mie len y frosas e suedentaño ala, Cila aral comaren bierns; tija quengó pizóncale, da edong
